<p><img src="https://www.oracle.com/a/ocom/img/dc/em/oracle-apex.png" width="200" align = "left"></p>
<p><img src="https://www.adaption-it.nl/wp-content/uploads/2019/02/Oracle-Logo.png.png" width="200" align = "right"></p>


### **<h1 align ="middle"><b> Running Object Detection using Yolov5 in APEX and Oracle Data Science! </b></h1>**

# **1.1 Create Custom Conda (as explained in the Blog)**

In [ ]:
### 1. create a custom conda environment. Paste and run the following command in a terminal
- odsc conda create -n object_detection_apex_conda -s object_detection_apex_conda -v 1.0

### 2. activate the newly created custom conda. Paste and run the following command in a terminal.
conda activate /home/datascience/conda/object_detection_apex_conda

---

> # **Important notes:**

> ### *1. Make sure to change your kernel to **"object_detection_apex_conda"** before running the below cells!*

> ### 2. The notebook assumes you are working in the home directory **(/home/datascience)**

> ### 3. The notebook uses a specific bucket name and namespace. Make sure to replace your bucket name and namespace each time you see **"YOUR_BUCKET_NAME"** and **"YOUR_NAMESPACE"** 

> ### 4. The images used are in the **".jpg"** format

---

# **1.2 Clone the Yolo5 github repo and perform installs**

### **1.2.1 Clone Yolov5 Github and install dependencies**

In [ ]:
!git clone https://github.com/ultralytics/yolov5 /home/datascience/conda/object_detection_apex_conda/yolov5 # this will clone the entire github repo in the same folder as your notebook is in.
!cd /home/datascience/conda/object_detection_apex_conda/yolov5 #this will step into the sub-directory yolov5
!pip install -r yolov5/requirements.txt  #this will install all the needed packages to run yolov5 

### **1.2.2 Additional installs**

In [ ]:
!conda install pytorch torchvision -c pytorch -y
!pip install opencv-python-headless
!python3 -m pip install oracle-ads[all-optional]
!pip install ipywidgets
!pip install ocifs

### **1.2.3 Test the Yolov5 on a sample image**

In [ ]:
import torch
from PIL import Image

#run Yolov5 model on a sample image
!python /home/datascience/conda/object_detection_apex_conda/yolov5/detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source /home/datascience/conda/object_detection_apex_conda/yolov5/data/images/zidane.jpg

#display the result. The below only works on the first run. Next runs are generated as exp2, exp3, exp4 and so on.
image = Image.open('/home/datascience/conda/object_detection_apex_conda/yolov5/runs/detect/exp/zidane.jpg')
image.show()

# **1.3 Publish Conda to Object Storage**

> **NOTE! Replace "YOUR_BUCKET_NAME" and "YOUR_NAMESPACE" with your own bucket name and namespace respectively in the below cell**

In [ ]:
#1. Establish connection between notebook session and your newly created bucket.
!odsc conda init -b <YOUR_BUCKET_NAME> -n <YOUR_NAMESPACE> -a resource_principal

In [ ]:
#2. Publish the 'object_detection_apex_conda' to the bucket. Make sure the bucket has no other custom conda environment, this will partly be overwritten
!odsc conda publish -s object_detection_apex_conda

# **2. Create Model artifacts and save to the Model Catalog**

## **2.1 Create Model artifacts**

In [ ]:
from ads.model.framework.tensorflow_model import TensorFlowModel
from ads.common.model_metadata import UseCaseType
from ads.common.model_artifact import ModelArtifact
from ads.common.model_export_util import prepare_generic_model
import os

> **NOTE! Replace "YOUR_BUCKET_NAME" and "YOUR_NAMESPACE" with your own bucket name and namespace respectively in the below cell**

In [ ]:
#1. Establish path to artifacts and conda slug
path_to_artifacts = '/home/datascience/model_artifacts'
conda_env = 'oci://<YOUR_BUCKET_NAME>@<YOUR_NAMESPACE>/conda_environments/cpu/object_detection_apex_conda/1/object_detection_apex_conda'

#2. create default artifacts
artifact = prepare_generic_model(path_to_artifacts, fn_artifact_files_included=False, force_overwrite=True, inference_conda_env=conda_env)

> **NOTE! Replace "YOUR_BUCKET_NAME" and "YOUR_NAMESPACE" with your own bucket name and namespace !!4 times!! respectively in the below cell**

In [ ]:
%%writefile "{path_to_artifacts}/score.py"
import os
import cv2
import sys
import torch
import numpy as np
import pandas as pd
import shutil
import ocifs
from ocifs import OCIFileSystem
import glob
import ads
import urllib
from yolov5 import models, utils 

def load_model():
    class DummyModel:
        def __init__(self):
            pass
    return DummyModel()

def predict(data, model=load_model()):
    
    x = data # this example does not use a json payload. But the JSON payload could be added to be processed as 'Data'
    
    #set authentication to resource principal
    ads.set_auth(auth='resource_principal')

    ########################### ########################### ########################### ########################### 
    ########################### 
    ########################### Input and output bucket
    ########################### 
    ########################### ########################### ########################### ########################### 
    
    input_bucket = "oci://<YOUR_BUCKET_NAME>@<YOUR_NAMESPACE>/input_image/"
    output_bucket = "oci://<YOUR_BUCKET_NAME>@<YOUR_NAMESPACE>/output_image/"
       
    ########################### ########################### ########################### ########################### 
    ########################### 
    ########################### Create local folder when the model is deployed
    ########################### 
    ########################### ########################### ########################### ########################### 

    #input image folder
    path_input_image_locally = "/home/datascience/input_image" 
    
    #delete folder when exists
    if os.path.exists(path_input_image_locally):
        shutil.rmtree(path_input_image_locally)
    
    #make as new folder
    if not os.path.exists(path_input_image_locally):         
        os.makedirs(path_input_image_locally)
    
    ## the same for output_image locally. Folder will be generated during model inference
    path_output_image_locally = "/home/datascience/output_image"
    
    #delete folder when exists
    if os.path.exists(path_output_image_locally):
        shutil.rmtree(path_output_image_locally)

    ########################### ########################### ########################### ########################### 
    ########################### 
    ########################### Copy input images from bucket to local folder ENV_INPUT_IMAGE
    ########################### 
    ########################### ########################### ########################### ########################### 

    fs = ocifs.OCIFileSystem()
    fs.invalidate_cache("oci://<YOUR_BUCKET_NAME>@<YOUR_NAMESPACE>/input_image/")
    fs.get("oci://<YOUR_BUCKET_NAME>@<YOUR_NAMESPACE>/input_image/*.jpg", "/home/datascience/input_image/")
    
    ########################### ########################### ########################### ########################### 
    ########################### 
    ########################### Run Yolov5 on the image
    ########################### 
    ########################### ########################### ########################### ########################### 
    
    ##get the image name
    for file in os.listdir(path_input_image_locally):
        if file.endswith(".jpg"):
            image_name = (os.path.join(path_input_image_locally, file))
            image_name_input = (file)
            
    #run the yolov5 on the image. Saves the output image to the "../output_image/" local folder    
    # Load model
    model = torch.hub.load("./yolov5", 'custom', path = './yolov5s.pt', source='local', force_reload=True)  #when loading from local directoy
   
    #model inference and save results to local folder
    results = model(image_name)
    results.save(save_dir=path_output_image_locally, exist_ok=False)

    #get the location of the output_image bucket and the image
    ENV_OUTPUT_IMAGE = os.environ.get("ENV_OUTPUT_IMAGE", output_bucket)
    local_output_image_path = os.path.join(path_output_image_locally + "/" + image_name_input)
    fs = OCIFileSystem()
    fs.invalidate_cache(input_bucket)
    
    print(local_output_image_path)

    #copy from the local folder to the bucket folder
    with open(local_output_image_path, 'rb') as f:
        with fs.open(ENV_OUTPUT_IMAGE + os.path.basename(local_output_image_path), 'wb') as file_out:
            file_out.write(f.read())
            
    #delete input image from the input bucket
    fs = ocifs.OCIFileSystem()
    fs.invalidate_cache(output_bucket)
    delete_object = os.path.join(input_bucket, image_name_input)
    fs.rm(delete_object, recursive=True)

    return "Your image has been processed and pushed to the output bucket!"

> **NOTE! Replace "YOUR_BUCKET_NAME" and "YOUR_NAMESPACE" with your own bucket name and namespace respectively in the below cell**

In [ ]:
%%writefile "{path_to_artifacts}/runtime.yaml"
# Model runtime environment
MODEL_ARTIFACT_VERSION: '3.0'
MODEL_DEPLOYMENT:
  INFERENCE_CONDA_ENV:
    INFERENCE_ENV_PATH: oci://<YOUR_BUCKET_NAME>@<YOUR_NAMESPACE>/conda_environments/cpu/object_detection_apex_conda/1/object_detection_apex_conda
    INFERENCE_PYTHON_VERSION: 3.8.12
MODEL_PROVENANCE:
  PROJECT_OCID: ocid1.datascienceproject.oc1.eu-frankfurt-1.amaaaaaangencdyaik5ssdqk4as2bhldxprh7vnqpk7yycsm7vymd344cgua
  TENANCY_OCID: ocid1.tenancy.oc1..aaaaaaaabu5fgingcjq3vc7djuwsdcutdxs4gsws6h4kfoldqpjuggxprgoa
  TRAINING_COMPARTMENT_OCID: ocid1.compartment.oc1..aaaaaaaae3n6r6hrjipbap2hojicrsvkzatrtlwvsyrpyjd7wjnw4za3m75q
  TRAINING_CONDA_ENV:
    TRAINING_ENV_PATH: oci://<YOUR_BUCKET_NAME>@<YOUR_NAMESPACE>/conda_environments/cpu/object_detection_apex_conda/1/object_detection_apex_conda
    TRAINING_ENV_SLUG: object_detection_apex_conda
    TRAINING_ENV_TYPE: published
    TRAINING_PYTHON_VERSION: 3.8.12
  TRAINING_REGION: eu-frankfurt-1
  TRAINING_RESOURCE_OCID: ocid1.datasciencenotebooksession.oc1.eu-frankfurt-1.amaaaaaangencdya3hrsuqabjsnnbwxegpvmo2r3xwl3bcwr2okp4fegouna
  USER_OCID: ocid1.saml2idp.oc1..aaaaaaaar3ydw5hoiob7dfjzoom2dvbhqkkd5fat6m7upe72emlsxhsfrbfa/bob.peulen@oracle.com
  VM_IMAGE_INTERNAL_ID: NB1313-DC257-VMP58-VMA1524-JL2.2.6

## **2.2 Copy the Yolov5 library into the model artifacts**

In [ ]:
#1. Copy the entire yolov5 folder into the model artifacts
!cp -R /home/datascience/conda/object_detection_apex_conda/yolov5 /home/datascience/model_artifacts

#2. When running step 1.2.3, the 'yolov5s.pt' file was created and added to your /home/datascience (./ or root) folder. We copy this .pt (the weights) in the model artifacts
!cp -R /home/datascience/yolov5s.pt /home/datascience/model_artifacts/

#3. Change access rights of the 'yolov5s.pt'. The instance running the deployed model needs to access it.
!chmod 777 /home/datascience/model_artifacts/yolov5s.pt

In [4]:
#4. Optional. You can check the access rights of the 'yolov5s.pt' file in the model_artifacts folder using the below. It should be "-rwxrwxrwx."
# !ls -l /home/datascience/model_artifacts/yolov5s.pt

## **2.3 Check all model artifacts**

In [ ]:
#all checks should be passed
artifact.introspect()

## **2.4 Save model artifacts to the Model Catalog**

In [ ]:
# Saving the model artifact to the model catalog. You can check the model added to your 'Models' in your Data Science Project.
catalog_entry = artifact.save(display_name='object_detection', description='object_detection', timeout=600)
catalog_entry.id

# **3. Create Model Deployment and test REST API**

In [ ]:
import oci 
from oci.data_science import DataScienceClient, DataScienceClientCompositeOperations

from oci.data_science.models import ModelConfigurationDetails, InstanceConfiguration, \
                                    FixedSizeScalingPolicy, CategoryLogDetails, LogDetails, \
                                    SingleModelDeploymentConfigurationDetails, CreateModelDeploymentDetails

auth = oci.auth.signers.get_resource_principals_signer()
data_science = DataScienceClient({}, signer=auth)

## **3.1 Model Deployment configuration**

In [ ]:
#You can change the display_name, description and instance_shape_name to your liking.
model_configuration_details_object = ModelConfigurationDetails(model_id=catalog_entry.id,
                                                               instance_configuration=InstanceConfiguration(instance_shape_name='VM.Standard2.1'),
                                                               scaling_policy=FixedSizeScalingPolicy(instance_count=1),
                                                               bandwidth_mbps=20)

single_model_config = SingleModelDeploymentConfigurationDetails(deployment_type='SINGLE_MODEL', model_configuration_details=model_configuration_details_object)

logs_configuration_details_object = {}

model_deploy_configuration = CreateModelDeploymentDetails(display_name='object_detection',
                                                          description='object_detection',
                                                          project_id=os.environ['PROJECT_OCID'],
                                                          compartment_id=os.environ['NB_SESSION_COMPARTMENT_OCID'],
                                                          model_deployment_configuration_details=single_model_config,
                                                          category_log_details=logs_configuration_details_object)

## **3.2 Creating a model deployment. This action takes a few minutes**

In [ ]:
data_science_composite = DataScienceClientCompositeOperations(data_science)
model_deployment = data_science_composite.create_model_deployment_and_wait_for_state(model_deploy_configuration, 
                                                                                     wait_for_states=["SUCCEEDED", "FAILED"])

## **3.3 Testing the deployed model using the REST API**

> # **!**

> #### Before you run the below cell, make sure to upload one .jpg image you would like to run Object Detection in the **"input_image"** bucket

> #### Fetch your HTTP Endpoint from your deployed model. In the Data Science Project, click on "Model Deployments". Click on your deployed model and select "Invoking your model". Copy and paste the HTTP endpoint from your deployed model in the below cell in uri = "<_YOUR_HTTP_ENDPOINT_>"

> # **!**

In [5]:
%%time
import requests
import oci
from oci.signer import Signer
import json

# REST API. In the below uri = f"", paste your HTTP Endpoint from the deployed model. Delete the short the example
uri = f"https://modeldeployment.eu-frankfurt-1.xxxxxxxxxxx/predict"

#dummy data, this could also be a JSON input. But the model expects some input.
data = "x"   

#authentication using resource principal
auth = oci.auth.signers.get_resource_principals_signer()

#make api call
response = requests.post(uri, json=data, auth=auth)
print(response)

<Response [200]>
CPU times: user 23.1 ms, sys: 5.87 ms, total: 29 ms
Wall time: 2.57 s


In [6]:
#review the response. Should be "Your image has been processed and pushed to the output bucket!"
response.content

b'"Your image has been processed and pushed to the output bucket!"'

In [ ]:
#the end